# Regression Week 2: Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using Turi Create. Now we will use Turi Create along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to a Turi Create SFrame to account for the intercept
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Fire up Turi Create

Make sure you have the latest version of Turi Create

In [37]:
import turicreate

from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('../data/home_data.sframe/')

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the other Week 2 notebook. For this notebook, however, we will work with the existing features.

In [5]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions) in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 

First we need to take the SFrame of our data and convert it into a 2D numpy array (also called a matrix). To do this we use Turi Create's built in .to_dataframe() which converts the SFrame into a Pandas (another python library) dataframe. We can then use Panda's .as_matrix() to convert the dataframe into a numpy matrix.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [6]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print(example_features[0,:]) # this accesses the first row of the data the ':' indicates 'all columns'
print(example_output[0]) # and the corresponding output

[1.00e+00 1.18e+03]
221900.0


# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrayws we can use np.dot() to compute this:

In [12]:
# example_features:numpy matrix, and example_output:numpy array
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point


predicted_value = np.dot(my_features, my_weights)
print('dot product:\n', my_weights, '.', my_features, ' =', predicted_value)

dot product:
 [1. 1.] . [1.00e+00 1.18e+03]  = 1181.0


np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [13]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

If you want to test your code run the following cell:

In [17]:
test_predictions = predict_output(example_features, my_weights)
print(test_predictions[0]) # should be 1181.0
print(test_predictions[1]) # should be 2571.0

1181.0
2571.0


In [20]:
print('0:', example_features[0], '.', my_weights, ' =', test_predictions[0])
print('1:', example_features[1], '.', my_weights, ' =', test_predictions[1])
print('10:', example_features[10], '.', my_weights, ' =', test_predictions[10])

0: [1.00e+00 1.18e+03] . [1. 1.]  = 1181.0
1: [1.00e+00 2.57e+03] . [1. 1.]  = 2571.0
10: [1.00e+00 3.56e+03] . [1. 1.]  = 3561.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [23]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2*np.dot(feature,errors)
    return(derivative)

To test your feature derivartive run the following:

In [31]:

# -------- 1. Get predictions based on example_features and example_output ---------
# example_features:numpy matrix, and example_output:numpy array, obtained above with: 
# (example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 

print('my weights are all 0, this makes all the predictions 0. For example:')
print('test predictions0:', example_features[0], '.', my_weights, ' =', test_predictions[0])
print('test predictions1:', example_features[1], '.', my_weights, ' =', test_predictions[1])
print('test predictions10:', example_features[10], '.', my_weights, ' =', test_predictions[10])

# -------- 2. Get the errors ---------
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output 
print('\n Prediction errors in this case is just the example_output: ')
print('output0:',example_output[0],'errors0',errors[0])
print('output1:',example_output[1],'errors1',errors[1])
print('output10:',example_output[10],'errors10',errors[10])


# -------- 2. Get the derivative ---------
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print('\nderivative with respect to w_0')
print(derivative)
print(-np.sum(example_output)*2) # should be the same as derivative

my weights are all 0, this makes all the predictions 0. For example:
test predictions0: [1.00e+00 1.18e+03] . [0. 0.]  = 0.0
test predictions1: [1.00e+00 2.57e+03] . [0. 0.]  = 0.0
test predictions10: [1.00e+00 3.56e+03] . [0. 0.]  = 0.0

 Prediction errors in this case is just the example_output: 
output0: 221900.0 errors0 -221900.0
output1: 538000.0 errors1 -538000.0
output10: 662500.0 errors10 -662500.0

derivative with respect to w_0
-23345850022.0
-23345850022.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [54]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        #predictions = predict_output(feature_matrix, weights)
        predictions = np.dot(feature_matrix, weights)
        print('\n predictions', predictions)
        
        # compute the errors as predictions - output
        errors = predictions - output
        print('errors', errors)
        
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        
        # **************** update each feature's weight **************
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            
            # ******* Update the ith weight ******
            # 1. compute the derivative for weight[i]:
            #deriv_weight_i = feature_derivative(errors, feature=feature_matrix[:,i])
            deriv_weight_i = 2*np.dot(feature_matrix[:,i],errors)
            
            # 2. subtract the step size times the derivative from the current weight
            weights[i] -= (step_size * deriv_weight_i)
            
            # ******* Asses convergence ******
            # Add the ith term for the magnitude of the gradient vector 
            #add the squared value of the derivative to the gradient sum of squares 
            gradient_sum_squares += (deriv_weight_i * deriv_weight_i)
            
            
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        print('gradient', gradient_sum_squares)
        print('gradient magnitude', gradient_magnitude)
        
        # determine convergence
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [33]:
train_data,test_data = sales.random_split(.8,seed=0)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimate the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [55]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)

initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Next run your gradient descent with the above parameters.

In [56]:
simple_weights = regression_gradient_descent(feature_matrix=simple_feature_matrix
                                      , output=output
                                      , initial_weights=initial_weights
                                      , step_size=step_size
                                      , tolerance=tolerance)


 predictions [-45820. -44430. -46230. ... -45470. -45400. -45980.]
errors [-267720. -582430. -226230. ... -405470. -445400. -370980.]
gradient 2.555457263634446e+27
gradient magnitude 50551530774393.43

 predictions [371735.75268253 864992.10740189 226242.87107465 ... 495936.99307949
 520777.24115888 314958.04278677]
errors [149835.75268253 326992.10740189  46242.87107465 ... 135936.99307949
 120777.24115888 -10041.95721323]
gradient 1.7232997037567737e+26
gradient magnitude 13127451023548.988

 predictions [263302.97905279 628829.24562979 155485.87883943 ... 355341.9670398
 373749.76463721 221228.01311587]
errors [  41402.97905279   90829.24562979  -24514.12116057 ...   -4658.0329602
  -26250.23536279 -103771.98688413]
gradient 1.1621254290688472e+25
gradient magnitude 3408996082527.5923

 predictions [291461.29400806 690157.08979222 173860.37582713 ... 391852.3217235
 411930.52726658 245568.25276672]
errors [ 69561.29400806 152157.08979222  -6139.62417287 ...  31852.3217235
  11930.

How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

In [54]:
simple_weights

array([-46999.88716555,    281.91211912])

Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [55]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [69]:
test_predictions = predict_output(feature_matrix=test_simple_feature_matrix, weights=simple_weights)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [70]:
test_predictions

array([356134.44317093, 784640.86422788, 435069.83652353, ...,
       663418.65300782, 604217.10799338, 240550.4743332 ])

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [71]:
def get_residual_sum_of_squares(predictions, output):
    # Then compute the residuals/errors
    residuals = predictions - output

    # Then square and add them up
    RSS = (residuals * residuals).sum()

    return(RSS)    

In [59]:
RSS = get_residual_sum_of_squares(predictions=test_predictions, output=test_output)
RSS

275400047593155.94

# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [57]:
model_features_2 = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix_2, output_2) = get_numpy_data(train_data, model_features_2, my_output)

initial_weights_2 = np.array([-100000., 1., 1.])
step_size_2 = 4e-12
tolerance_2 = 1e9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [58]:
weights_model2 = regression_gradient_descent(feature_matrix=feature_matrix_2
                                      , output=output_2
                                      , initial_weights=initial_weights_2
                                      , step_size=step_size_2
                                      , tolerance=tolerance_2)


 predictions [-97480. -95740. -96510. ... -96940. -96990. -97960.]
errors [-319380. -633740. -276510. ... -456940. -496990. -422960.]
gradient 5.339520187093092e+27
gradient magnitude 73072020548860.52

 predictions [421002.96577892 792804.3785608  603427.43848423 ... 534683.2867589
 526304.56305387 323122.22062344]
errors [199102.96577892 254804.3785608  423427.43848423 ... 174683.2867589
 126304.56305387  -1877.77937656]
gradient 5.140749489327198e+26
gradient magnitude 22673220965110.355

 predictions [259518.53331584 519010.81199792 380998.2189088  ... 338458.60344862
 333152.72488025 192305.75558163]
errors [  37618.53331584  -18989.18800208  200998.2189088  ...  -21541.39655138
  -66847.27511975 -132694.24441837]
gradient 4.985482013056992e+25
gradient magnitude 7060794582096.97

 predictions [308947.89676147 605683.9180731  444775.73350445 ... 399005.83731368
 393224.89063921 232670.58074683]
errors [ 87047.89676147  67683.9180731  264775.73350445 ...  39005.83731368
  -6775.10

 384132.36127082 216594.76066407]
errors [  54871.92373882  101860.74774254   87152.55918614 ...   14892.1252051
  -15867.63872918 -108405.23933593]
gradient 2.0501615245926294e+19
gradient magnitude 4527870939.627839

 predictions [276768.54872413 639869.76375662 267125.76844926 ... 374890.51303747
 384133.20666851 216593.68588251]
errors [  54868.54872413  101869.76375662   87125.76844926 ...   14890.51303747
  -15866.79333149 -108406.31411749]
gradient 1.9540301612614025e+19
gradient magnitude 4420441336.859254

 predictions [276765.25378601 639878.56585406 267099.61335736 ... 374888.93912057
 384134.03200801 216592.6366015 ]
errors [  54865.25378601  101878.56585406   87099.61335736 ...   14888.93912057
  -15865.96799199 -108407.3633985 ]
gradient 1.862406364447247e+19
gradient magnitude 4315560640.805836

 predictions [276762.03702452 639887.15911031 267074.07882894 ... 374887.40254687
 384134.83776522 216591.61221598]
errors [  54862.03702452  101887.15911031   87074.07882894 ...

Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [59]:
(test_feature_matrix_2, test_output_2) = get_numpy_data(test_data, model_features_2, my_output)

In [60]:
test_predictions_2 = predict_output(feature_matrix=test_feature_matrix_2, weights=weights_model2)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [75]:
test_predictions_2

array([366651.41203656, 762662.39786164, 386312.09499712, ...,
       682087.39928241, 585579.27865729, 216559.20396617])

What is the actual price for the 1st house in the test data set?

In [76]:
test_output_2

array([310000., 650000., 233000., ..., 610685., 400000., 402101.])

**Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**

model 1

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [77]:
RSS2 = get_residual_sum_of_squares(predictions=test_predictions_2, output=test_output_2)
RSS2

270263446465244.06

**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **

model 1: 275400047593155.94 <br>
model 2: 270263446465244.06

lowest: model 2